# SPD

Replicate the design of [Smaldino et al. (2013)](https://www.journals.uchicago.edu/doi/10.1086/669615).

Add per-agent memory. Remember defectors. Vary memory size.

## Imports

In [ ]:
# model
from mesa_fork import Model, Agent
from mesa_fork.time import RandomActivation
from mesa_fork.space import SingleGrid
from mesa_fork.datacollection import DataCollector
from enum import Enum

# visualization
import numpy as np
import matplotlib.pyplot as plt
plt.rc('axes', labelsize=8)
%matplotlib inline
plt.style.use('seaborn')
import holoviews as hv
%load_ext holoviews.ipython
import seaborn as sns
sns.set_theme(style="ticks")

from my_plot import my_plot_export

# parameter sweep
from mesa_fork.batchrunner import BatchRunnerMP

## Setup model

In [ ]:
class Memory:
    """
    Fixed-size FIFO (first-in-first-out) memory.
    Used by agents to remeber the most recent defectors.
    """
    
    def __init__(self,
                 size):
        """
        Args:
            size: memory size
        """
        
        self.size = size
        self.memory = []
        
        
    def add(self, item):
        
        # remove duplicates
        self.memory[:] = list(filter(
            lambda x: x != item,
            self.memory))
        
        self.memory.insert(0, item)
        
        # truncate to size
        while len(self.memory) > self.size:
            self.memory.pop(-1)
       
    
    def contains(self, item):
        try:
            self.memory.index(item)
            return True
        except ValueError:
            return False
            

In [ ]:
class Action(Enum):
    COOPERATE = 1
    DEFECT    = 2

    
class SmaldinoAgentWithMemory(Agent):
    
    def __init__(self, 
                 model,
                 energy,
                 max_energy,
                 memory_size,
                 cooperator=False):
        
        super().__init__(model.next_id(), model)
        
        self.memory = Memory(memory_size)
        
        self.energy = energy
        self.max_energy = max_energy
        self.cooperator = cooperator
        
        self.played = True
        self.newborn = True
        
        
    def play(self):
        """
        Return game action based on agent type.
        All agents use either always-cooperate or always-defect strategy.
        """
        
        if self.cooperator:
            return Action.COOPERATE
        else:
            return Action.DEFECT
        
        
    def step(self):
        """
        Agent behaviour in a single timestep.
        """
        
        # don't step if created this turn
        if self.newborn:
            return

        neighbors = self.model.grid.get_neighbors(self.pos, moore=True)
        # discard neighbors who have already played a game in this step
        opponents = list(filter(
            lambda a: not a.played,
            neighbors))
        # discard remembered defectors
        opponents = list(filter(
            lambda a: not self.memory.contains(a.unique_id) and not a.memory.contains(self.unique_id),
            opponents))

        # find opponent
        if opponents:
            opponent = self.random.choice(opponents)

            # play pd game
            if not self.played:

                a = self.play()
                b = opponent.play()

                R, T, S, P = self.model.R, self.model.T, self.model.S, self.model.P

                if   a == Action.COOPERATE and b == Action.COOPERATE:
                    self.energy     += R
                    opponent.energy += R
                    
                elif a == Action.COOPERATE and b == Action.DEFECT:
                    self.energy     += S
                    opponent.energy += T

                    # remember betrayal
                    self.memory.add(opponent.unique_id)
                    
                elif a == Action.DEFECT    and b == Action.COOPERATE:
                    self.energy     += T
                    opponent.energy += S

                    # remember betrayal
                    opponent.memory.add(self.unique_id)
                    
                elif a == Action.DEFECT    and b == Action.DEFECT:
                    self.energy     += P
                    opponent.energy += P

                    # remember betrayals
                    self.memory.add(opponent.unique_id)
                    opponent.memory.add(self.unique_id)
                    
                self.energy = min(self.energy, self.max_energy)
                opponent.energy = min(opponent.energy, opponent.max_energy)

                self.played = True
                opponent.played = True

            # reproduce
            max_population = (self.model.grid.width * self.model.grid.height) / 2
            if (    self.cooperator and self.model.last_cooperator_count < max_population) or \
               (not self.cooperator and self.model.last_defector_count   < max_population):

                neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True)
                unoccupied = list(filter(
                    lambda c: self.model.grid.is_cell_empty(c), 
                    neighborhood))

                if self.energy >= (self.model.energy_to_reproduce * 2) and unoccupied:

                    cell = self.random.choice(unoccupied)
                    offspring = SmaldinoAgentWithMemory(self.model,
                                                        energy=self.model.energy_to_reproduce,
                                                        max_energy=self.max_energy,
                                                        cooperator=self.cooperator,
                                                        memory_size=self.memory.size)
                    self.model.grid.position_agent(offspring, cell[0], cell[1])
                    self.model.schedule.add(offspring)

                    # update values for DataCollector
                    self.model.agent_count += 1
                    if self.cooperator:
                        self.model.cooperator_count += 1
                    else:
                        self.model.defector_count += 1

                    self.energy -= self.model.energy_to_reproduce


        elif not self.played:
            # attempt movement
            neighborhood = self.model.grid.get_neighborhood(self.pos, moore=True)
            unoccupied = list(filter(
                lambda c: self.model.grid.is_cell_empty(c), 
                neighborhood))

            if unoccupied:
                cell = self.random.choice(unoccupied)
                self.model.grid.move_agent(self, cell)


        # energy deduction (cost of living)
        self.energy -= self.model.living_cost
        if self.energy <= 0:
            # die
            self.model.grid.remove_agent(self)
            self.model.schedule.remove(self)
            return
            
        # update values for DataCollector
        self.model.agent_count += 1

        if self.cooperator:
            self.model.cooperator_count += 1
        else:
            self.model.defector_count += 1


In [ ]:
class SPDModel(Model):
    
    def __init__(self,
                 R=3, T=5, S=-1, P=0,
                 starting_agent_count=10,
                 starting_energies=range(1,51),
                 max_energy=150,
                 energy_to_reproduce=50,
                 living_cost=1,
                 memory_size=0,
                 grid_size=10,
                 wrap=True):
        """
        Smaldino's spatial prisonner's dilemma model
        extended with limited memory
        
        Args:
            R, T, S, P:            PD payoffs
            starting_agent_count:  starting number of agents
            starting_energies:     list of possible starting energies for agents (picked at random)
            max_energy:            maximal energy an agent can hold
            energy_to_reproduce:   energy required to reproduce
            living_cost:           energy deducted at the end of each step
            memory_size:           agent memory size
            grid_size:             size length of square grid to use
            wrap:                  whether to wrap grid (torus bounds)
        """
        
        super().__init__()
        self.schedule = RandomActivation(self)
        self.grid = SingleGrid(grid_size, grid_size, torus=wrap)
        
        self.R = R
        self.T = T
        self.S = S
        self.P = P
        self.energy_to_reproduce = energy_to_reproduce
        self.living_cost = living_cost
        
        # Setup agents
        self.cooperator_count = 0
        self.defector_count = 0

        for i in range(starting_agent_count):
            energy = self.random.choice(starting_energies)
            cooperator = i%2 == 0
            
            if cooperator:
                self.cooperator_count += 1
            else:
                self.defector_count += 1
            
            agent = SmaldinoAgentWithMemory(self, 
                                            energy,
                                            max_energy=max_energy,
                                            cooperator=cooperator,
                                            memory_size=memory_size)
        
            cell = self.random.choice(list(self.grid.empties))

            self.grid.position_agent(agent, cell[0], cell[1])
            self.schedule.add(agent)
        
        self.agent_count = starting_agent_count
        
        # Init model
        self.running = True
        
        self.datacollector = DataCollector(
            {
                "agent_count": "agent_count",
                "cooperator_count": "cooperator_count",
                "defector_count": "defector_count",
            },
        )
        self.datacollector.collect(self)
        
        
    def step(self):
        
        # setup for step
        self.last_cooperator_count = self.cooperator_count
        self.last_defector_count   = self.defector_count
        
        self.agent_count = 0
        self.cooperator_count = 0
        self.defector_count = 0

        for a in self.schedule.agents:
            a.played = False
            a.newborn = False
    
        # step
        self.schedule.step()
        self.datacollector.collect(self)
        
        # stop the model if no agents are alive
        if self.cooperator_count == 0 or self.defector_count == 0:
            self.running = False


## Run model

In [ ]:
spd = SPDModel(R=3, T=5, S=-1, P=0,
               starting_agent_count=64,
               starting_energies=range(1,50),
               max_energy=150,
               energy_to_reproduce=50,
               living_cost=1,
               memory_size=5,
               grid_size=20,
               wrap=True)

In [ ]:
i = 0
while spd.running and i < 50000:
    spd.step()
    i += 1

### Render visualization

In [ ]:
def value(cell):
    if cell is None:
        return 0
    elif isinstance(cell, Agent):
        if cell.cooperator:
            return 2
        else:
            return 10
    else:
        raise Exception("Unidentified cell: {}".format(cell))
        
hmap = hv.HoloMap(kdims='step')
i = 0
while spd.running and i < 100:
    spd.step()
    data = np.array([[value(c) for c in row] for row in spd.grid.grid])
    hmap[i] = hv.Image(data, vdims=[hv.Dimension('State', range=(0,10))])
    i += 1
hmap

### Check results

In [ ]:
results = spd.datacollector.get_model_vars_dataframe()

fig, ax = plt.subplots(1, 1)
sns.lineplot(data=results[['cooperator_count', 'defector_count']], ax=ax)
# ax.set_xlim(0, 1000)
ax.set_ylim(0)
ax.set_xlabel('step')
ax.set_ylabel('number of agents of a type')

In [ ]:
my_plot_export(fig, [ax], 'frequency_time_with_memory')

## Paramater sweep

In [ ]:
variable_parameters = {
    "memory_size": range(0, 4),
}
fixed_parameters = {
    "R": 3,
    "T": 5,
    "S": -1,
    "P": 0,
    "starting_agent_count":  64,
    "starting_energies":     range(1,50),
    "max_energy":            150,
    "energy_to_reproduce":   50,
    "living_cost":           1,
    "grid_size":             20,
    "wrap":                  True,
}

iterations = 20
max_steps = 50

param_run = BatchRunnerMP(SPDModel,
                          nr_processes=None,  # detect automatically
                          variable_parameters=variable_parameters,
                          fixed_parameters=fixed_parameters,
                          iterations=iterations,
                          max_steps=max_steps,
                          model_reporters={
                              "agent_count": lambda m: m.agent_count,
                              "cooperator_count": lambda m: m.cooperator_count,
                              "defector_count": lambda m: m.defector_count,
                          })

param_run.run_all()

In [ ]:
run_data = param_run.get_model_vars_dataframe()
run_data['cooperator_frequency'] = (run_data['cooperator_count'] / run_data['agent_count'])
run_data['defector_frequency']   = (run_data['defector_count']   / run_data['agent_count'])
run_data = run_data.dropna()
run_data.head()

In [ ]:
fig, ax = plt.subplots(nrows=1)
sns.boxplot(x="memory_size", y="cooperator_frequency", data=run_data, ax=ax)
ax.set_ylim(0, 1.0)
ax.set_xlabel("Memory size")
ax.set_ylabel("Cooperator frequency")
ax.set_title("After {} steps".format(max_steps))

In [ ]:
my_plot_export(fig, [ax], 'cooperator_frequency_memory_{}steps'.format(max_steps))
my_plot_export(fig, [ax], 'cooperator_frequency_memory_{}steps_large'.format(max_steps), fontsize=16)